In [1]:
import cv2, os, json, math, time, tqdm
import pandas as pd
import numpy as np
from collections import Counter, OrderedDict, defaultdict
from zipfile import ZipFile
from pycocotools import mask as maskUtils

## Deepfashion2 to COCO 

In [2]:
def df2coco(path_annos_image:str, num_images:int, output_dir:str):
    """Convert DF2 annotation format to COCO.
    Args:
        path_annos_image: (str) path of annos and image folders
        num_images: (int) number of images == number of annos files
        output_dir: (str) path where to save new json
    
    :return: json file with COCO annotations
    """
    dataset = {
        "info": {},
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }

    ### Categories
    keypoints = [str(i) for i in range(1,295)]
    categories = [[1, 'short_sleeved_shirt'], 
             [2, 'long_sleeved_shirt'],
             [3, 'short_sleeved_outwear'],
             [4, 'long_sleeved_outwear'],
             [5, 'vest'],
             [6, 'sling'],
             [7, 'shorts'],
             [8, 'trousers'],
             [9, 'skirt'],
             [10, 'short_sleeved_dress'],
             [11, 'long_sleeved_dress'],
             [12, 'vest_dress'],
             [13, 'sling_dress']]

    dataset['categories'] = [{'id': categories[i][0], 
                             'name': categories[i][1],
                             'supercategory': 'clothes',
                             'keypoints': keypoints,
                             'skeleton': []} for i in range(len(categories))]
    
    assert len(os.listdir(path_annos_image+'/image')) == len(os.listdir(path_annos_image+'/annos')) == num_images 

    ### Images / Annotations
    sub_index = 0 # the index of ground truth instance
    for num in tqdm.tqdm(range(1, num_images+1)):
        json_name = path_annos_image+'/annos/' + str(num).zfill(6)+'.json'
        image_name = path_annos_image+'/image/' + str(num).zfill(6)+'.jpg'

        if (num>=0):
            shp = cv2.imread(image_name).shape
            width, height = shp[1], shp[0]
            temp = json.load(open(json_name))
            pair_id = temp['pair_id']
            source = temp['source']

            dataset['images'].append({
                'coco_url': '',
                'date_captured': '',
                'file_name': str(num).zfill(6) + '.jpg',
                'flickr_url': '',
                'id': num,
                'license': 0,
                'width': width,
                'height': height,
                'pair_id': pair_id,
                'source': source})

            for i in temp.keys(): # for each item 
                if i == 'source' or i=='pair_id':
                    continue
                else:
                    points = np.zeros(294 * 3)
                    sub_index = sub_index + 1
                    box = temp[i]['bounding_box']
                    w = box[2]-box[0]
                    h = box[3]-box[1]
                    x_1 = box[0]
                    y_1 = box[1]
                    bbox = [x_1,y_1,w,h]
                    cat = temp[i]['category_id']
                    seg = temp[i]['segmentation']

                    style = temp[i]['style']
                    scale = temp[i]['scale']
                    occlusion = temp[i]['occlusion']
                    viewpoint = temp[i]['viewpoint']
                    zoom_in = temp[i]['zoom_in']

                    landmarks = temp[i]['landmarks']
                    points_x = landmarks[0::3]
                    points_y = landmarks[1::3]
                    points_v = landmarks[2::3]
                    points_x = np.array(points_x)
                    points_y = np.array(points_y)
                    points_v = np.array(points_v)

                    if cat == 1:
                        for n in range(0, 25):
                            points[3 * n] = points_x[n]
                            points[3 * n + 1] = points_y[n]
                            points[3 * n + 2] = points_v[n]
                    elif cat ==2:
                        for n in range(25, 58):
                            points[3 * n] = points_x[n - 25]
                            points[3 * n + 1] = points_y[n - 25]
                            points[3 * n + 2] = points_v[n - 25]
                    elif cat ==3:
                        for n in range(58, 89):
                            points[3 * n] = points_x[n - 58]
                            points[3 * n + 1] = points_y[n - 58]
                            points[3 * n + 2] = points_v[n - 58]
                    elif cat == 4:
                        for n in range(89, 128):
                            points[3 * n] = points_x[n - 89]
                            points[3 * n + 1] = points_y[n - 89]
                            points[3 * n + 2] = points_v[n - 89]
                    elif cat == 5:
                        for n in range(128, 143):
                            points[3 * n] = points_x[n - 128]
                            points[3 * n + 1] = points_y[n - 128]
                            points[3 * n + 2] = points_v[n - 128]
                    elif cat == 6:
                        for n in range(143, 158):
                            points[3 * n] = points_x[n - 143]
                            points[3 * n + 1] = points_y[n - 143]
                            points[3 * n + 2] = points_v[n - 143]
                    elif cat == 7:
                        for n in range(158, 168):
                            points[3 * n] = points_x[n - 158]
                            points[3 * n + 1] = points_y[n - 158]
                            points[3 * n + 2] = points_v[n - 158]
                    elif cat == 8:
                        for n in range(168, 182):
                            points[3 * n] = points_x[n - 168]
                            points[3 * n + 1] = points_y[n - 168]
                            points[3 * n + 2] = points_v[n - 168]
                    elif cat == 9:
                        for n in range(182, 190):
                            points[3 * n] = points_x[n - 182]
                            points[3 * n + 1] = points_y[n - 182]
                            points[3 * n + 2] = points_v[n - 182]
                    elif cat == 10:
                        for n in range(190, 219):
                            points[3 * n] = points_x[n - 190]
                            points[3 * n + 1] = points_y[n - 190]
                            points[3 * n + 2] = points_v[n - 190]
                    elif cat == 11:
                        for n in range(219, 256):
                            points[3 * n] = points_x[n - 219]
                            points[3 * n + 1] = points_y[n - 219]
                            points[3 * n + 2] = points_v[n - 219]
                    elif cat == 12:
                        for n in range(256, 275):
                            points[3 * n] = points_x[n - 256]
                            points[3 * n + 1] = points_y[n - 256]
                            points[3 * n + 2] = points_v[n - 256]
                    elif cat == 13:
                        for n in range(275, 294):
                            points[3 * n] = points_x[n - 275]
                            points[3 * n + 1] = points_y[n - 275]
                            points[3 * n + 2] = points_v[n - 275]
                    num_points = len(np.where(points_v > 0)[0])

                    dataset['annotations'].append({
                        'image_id': num,
                        'category_id': cat,
                        'id': sub_index,

                        'bbox': bbox,
                        'area': w*h,
                        'segmentation': seg,

                        'iscrowd': 0,
                        'num_keypoints':num_points,
                        'keypoints':points.tolist(),

                        'style': style,
                        'scale': scale, 
                        'occlusion': occlusion,
                        'viewpoint': viewpoint,
                        'zoom_in': zoom_in
                    })

 
    with open(output_dir, 'w') as f:
        json.dump(dataset, f)

In [6]:
DF2_DATA_DIR = '.../datasets/deepfashion2' #@@@ OVERRIDE: Path of folder with DF2 annotations

# CHECKS
# Check number of images = number of annotation files
images_names_tr = sorted(os.listdir(os.path.join(DF2_DATA_DIR, 'train/image'))) 
annos_names_tr = sorted(os.listdir(os.path.join(DF2_DATA_DIR, 'train/annos')))
assert len(images_names_tr) == len(annos_names_tr)

images_names_val = sorted(os.listdir(os.path.join(DF2_DATA_DIR, 'validation/image'))) 
annos_names_val = sorted(os.listdir(os.path.join(DF2_DATA_DIR, 'validation/annos')))
assert len(images_names_val) == len(annos_names_val)

# Check there is 1 and only 1 annotation file for each immage
for i in range(len(images_names_tr)):
    assert annos_names_tr[i].split('.')[0] == images_names_tr[i].split('.')[0]
    
for i in range(len(images_names_val)):
    assert annos_names_val[i].split('.')[0] == images_names_val[i].split('.')[0]  

# CONVERT!
df2coco(os.path.join(DF2_DATA_DIR, 'train'), len(images_names_tr), 'df2_annos_tr.json')
df2coco(os.path.join(DF2_DATA_DIR, 'validation'), len(images_names_val), 'df2_annos_val.json')

## VGG Image Annotator to COCO 

In [2]:
def via2coco(via_json_path, ontology_path, img_path, source, output_dir):
    """Convert a json from VGG Image Annotator (VIA) format to COCO.
    Args:
        via_json_path: (str) path of VIA json file
        ontology_path: (str) path of xlsx file with categories
        source: (str) source of images (user/shop/IG/user_shop)
        img_path: (str) path of folder with images
        output_dir: (str) path where to save new json
    
    :return: json file with COCO annotations
    """
    via = json.load(open(via_json_path, encoding="utf-8"))
    ont_df = pd.read_excel(ontology_path, sheet_name=0, engine="openpyxl")

    coco = {
        "info": {},
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }

    coco['info'] = {
        "description": via['_via_settings']['project']['name'],
        "version": "1.0",
        "VIA_version": via['_via_data_format_version'],
        "year": 2022,
        "date_created": time.asctime(time.localtime(time.time()))
    }

    via_cats = via['_via_attributes']['region']['clothing']['options']
    for k,v in tqdm.tqdm(via_cats.items()):
        rows, cols = np.where(ont_df == v)
        if ont_df.columns[cols[0]-1] == 'category':
            cat = ont_df.iloc[rows[0], cols[0]-1]
            sup = ont_df.iloc[rows[0], cols[0]-2]
        else:   
            cat, sup = '', ont_df.iloc[rows[0], cols[0]-1]
        c = {
            'id': int(k), 
            'name': v.strip().replace(' ', '_').lower(),
            'category': cat,
            'supercategory': sup}
        coco['categories'].append(c)
    
    id = 0
    for k,v in tqdm.tqdm(via['_via_img_metadata'].items()):
        if 'clothing' in v['regions'][0]['region_attributes'].keys():
            # Images
            k = int(v['filename'][:-4])
            assert int(k) == int(v['filename'][:-4])
            shp = cv2.imread(os.path.join(img_path, v['filename'])).shape
            img = {
                "id": int(k),
                "file_name": v['filename'],
                "width": shp[1],
                "height": shp[0],
                "source" : source, 
                "people" : list(v['file_attributes']['people'])}
            coco['images'].append(img)

            # Annotations
            for region in v['regions']:
                cat_id = int(region['region_attributes']['clothing'])
                assert cat_id in (d['id'] for d in coco['categories'])
                points_x = region['shape_attributes']['all_points_x']
                points_y = region['shape_attributes']['all_points_y']

                seg = [] # There will be just 1 polygon for 1 instance in VIA format
                for x, y in zip(points_x, points_y):
                    seg.append(x)
                    seg.append(y)
                segmentation = [seg]

                rles = maskUtils.frPyObjects(segmentation, shp[0], shp[1])
                rle = maskUtils.merge(rles) # If a single object consist of multiple parts, merge all parts into 1 RLE
                area = maskUtils.area(rle).astype(float)
                bbox = list(maskUtils.toBbox(rle))
                
                ann = {
                    "id": id,
                    "image_id": int(k),
                    "category_id": cat_id, # int
                    "iscrowd": 0,
                    "area": area,  # float
                    "bbox": bbox,  # [x,y,w,h]
                    "segmentation": segmentation  # [x1,y1,x2,y2,...]
                }
                id += 1
                coco['annotations'].append(ann)
    
    with open(output_dir, 'w') as f:
        json.dump(coco, f)
    return coco

In [ ]:
#@@@ OVERRIDE
DIR_VIA_JSON = '.../via.json' # Path of json file with the VIA annotations
DIR_XLSX_ONTOLOGY = './tools/fashion_ontology.xlsx' # Path of excel file with fashion ontolgy
DIR_IMAGES = '.../images' # Path of folder with images (whose annotations have been saved in the VIA json file)

coco = via2coco(DIR_VIA_JSON, DIR_XLSX_ONTOLOGY, DIR_IMAGES, '', './coco.json')